In [1]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.autograd import Variable
from torch import optim
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import math
import copy
from torch.optim.lr_scheduler import MultiStepLR
from torchvision.models import resnet50, resnet34, resnet18, wide_resnet50_2, ResNet50_Weights, alexnet
import gc
import os
import pandas as pd
from torchvision.io import read_image
from flax.training import checkpoints
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor


!pip install wandb
!pip install kornia
!pip install optuna
import optuna
import kornia
import wandb

In [2]:
from networks import ConvNet, AlexNet
from distill import ParamDiffAug
from utils import evaluate_synset, get_network
import argparse
#labels_train = torch.load('/datasets/mtt-cifar10-50-ipc/cifar10_50ipc_labels.pt')
#images_train = torch.load('/datasets/mtt-cifar10-50-ipc/cifar10_50ipc_images.pt')
labels_train = torch.load('./data/cifar100_10ipc_labels.pt')
images_train = torch.load('./data/cifar100_10ipc_images.pt')

In [3]:
batch_size = 256
train_dataset = torchvision.datasets.CIFAR100(root = './data',
                                                    train = True,
                                                    transform = transforms.Compose([
                                                            transforms.ToTensor(),
                                                            transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2435, 0.2616]),]),
                                                    download=True)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                                    batch_size = batch_size,
                                                    shuffle = True)


test_dataset = torchvision.datasets.CIFAR100(root = './data',
                                                    train = False,
                                                    transform = transforms.Compose([
                                                            transforms.ToTensor(),
                                                            transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2435, 0.2616]),]),
                                                    download=True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                                    batch_size = batch_size,
                                                    shuffle = True)

transform_train = transforms.Compose([transforms.Resize((32,32)),  #resises the image so it can be perfect for our model.
                                      transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
                                      transforms.RandomRotation(10),     #Rotates the image to a specified angel
                                      transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
                                      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Set the color params
                                      transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
                                      transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2435, 0.2616]), #Normalize all the images
                               ])


augmented_train_dataset = torchvision.datasets.CIFAR100(root = './data',
                                                    train = True,
                                                    transform = transform_train,
                                                    download=True)

augmented_train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                                    batch_size = batch_size,
                                                    shuffle = True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [10]:
class RandomCIFAR10Dataset(Dataset):
    def __init__(self, root, transform=None, target_transform=None, num_samples=1000):
        self.cifar = torchvision.datasets.CIFAR10(root, train=True, download=True, transform=transform, target_transform=target_transform)
        self.num_samples = num_samples
        self.indices = torch.randperm(len(self.cifar))[:self.num_samples]

    def __getitem__(self, index):
        return self.cifar[self.indices[index]]

    def __len__(self):
        return self.num_samples
    
class EasyHardCIFAR10Dataset(Dataset):
    def __init__(self, root, pretrained_model, transform=None, target_transform=None, mode='easy', num_samples=1000):
        assert mode in ['easy', 'hard'], "Mode should be 'easy' or 'hard'"
        self.cifar = CIFAR10(root, train=True, download=True, transform=transform, target_transform=target_transform)
        self.pretrained_model = pretrained_model
        self.mode = mode
        self.num_samples = num_samples

        self.pretrained_model.eval()
        with torch.no_grad():
            data, targets = zip(*[(data, target) for data, target in self.cifar])
            data = torch.stack(data)
            targets = torch.tensor(targets)
            outputs = self.pretrained_model(data)
            _, preds = torch.max(outputs, 1)
            probs = torch.nn.functional.softmax(outputs, dim=1)

            # Get confidence scores and sort them
            confidence_scores, indices = torch.sort(probs.max(dim=1).values, descending=True)
            sorted_preds = preds[indices]
            sorted_targets = targets[indices]

            if mode == 'easy':
                mask = sorted_preds == sorted_targets
            else:  # mode == 'hard'
                mask = sorted_preds != sorted_targets

            # Select num_samples many samples
            self.indices = indices[mask][:self.num_samples].tolist()

    def __getitem__(self, index):
        return self.cifar[self.indices[index]]

    def __len__(self):
        return len(self.indices)

#Get pretrained model to determine what is easy/hard
model = AlexNet(3,10).to('cuda')
args = argparse.Namespace(lr_net='.01', device='cuda', epoch_eval_train='1000',batch_train=batch_size,dataset='cifar10',dsa=True,dsa_strategy='color_crop_cutout_flip_scale_rotate',dsa_param = ParamDiffAug(), dc_aug_param=None, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)) #, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)
pretrained_model, acc_train_list, acc_test = evaluate_synset(7, model,images_train,labels_train,test_loader,args)

easy_train_dataset = EasyHardCIFAR10Dataset(root = './data', pretrained_model = pretrained_model, transform=None, target_transform=None, mode='easy', num_samples=1000)

easy_train_loader = torch.utils.data.DataLoader(dataset = easy_train_dataset,
                                                    batch_size = batch_size,
                                                    shuffle = True)

hard_train_dataset = EasyHardCIFAR10Dataset(root = './data', pretrained_model = pretrained_model, transform=None, target_transform=None, mode='hard', num_samples=1000)

hard_train_loader = torch.utils.data.DataLoader(dataset = hard_train_dataset,
                                                    batch_size = batch_size,
                                                    shuffle = True)

random_train_dataset = RandomCIFAR10Dataset(root = './data', transform=None, target_transform=None, num_samples=1000)

random_train_loader = torch.utils.data.DataLoader(dataset = random_train_dataset,
                                                    batch_size = batch_size,
                                                    shuffle = True)

NameError: name 'batch_size' is not defined

In [4]:
def train(model,train_loader, num_epochs, lr = .0008, weight_decay = .0008, gamma = .15, milestones = [50,65,80]):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay = weight_decay)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    cost = nn.CrossEntropyLoss()
    scheduler = MultiStepLR(optimizer, milestones=milestones, gamma= gamma)
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):  
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            
            loss = cost(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        scheduler.step()
            
def test(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    # Test the model
    model.eval()
    model.to(device)
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(test_loader): 
            images, labels = images.to(device), labels.to(device)
            test_output = model(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            correct += (pred_y == labels).sum().item()
            total += labels.size(0)
        accuracy = correct / total

    print('Test Accuracy:', accuracy)
    return accuracy

def get_parameters_to_prune(model):
    parameters_to_prune = []
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, 'weight'))
    return tuple(parameters_to_prune)

def sparsity_print(model):
    prune.global_unstructured(get_parameters_to_prune(model),pruning_method=prune.L1Unstructured,amount=0)
    zero = total = 0
    for module, _ in get_parameters_to_prune(model):
        zero += float(torch.sum(module.weight == 0))
        total += float(module.weight.nelement())
    print('Number of Zero Weights:', zero)
    print('Total Number of Weights:', total)
    print('Sparsity', zero/total)
    #TODO: Implement Node Sparsity
    return zero, total

#Standard IMP with Weight Rewinding, name is a string that allows us to save models/logs appropriately, path is the location of folder we save to, start_iter should be 0 but if a experiment stops halfway through it allows us to begin there,
#amount = % params pruned each pruning iteration, save_model downloads each model at every iter, reinit is boolean value if we want to test results on reinitialized weights
#reinit_model is the specific model that holds the reinitialized weights
def LotteryTicketRewinding(model, name, path, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs = 60, k = 1, amount = .2, save_model = True, seed = 0, reinit = False, reinit_model = None):
    torch.manual_seed(seed)
    zeros = []
    totals = []
    acc = []
    reinit_acc = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    
    #Create Rewind Weights after training K epochs
    train(model, train_loader,num_epochs = k)
    torch.save(model.state_dict(), path + name + '_RewindWeights' + '_' + str(k))
    model_rewind = copy.deepcopy(model).to(device)
    
    #Finish off the pretraining
    train(model, train_loader,num_epochs = num_epochs - k)

    #Lottery Ticket Rewinding: Prune, Rewind, Train
    for i in range(start_iter,end_iter):
        print('LTR Iteration:', i+1)
        #Prune
        prune.global_unstructured(get_parameters_to_prune(model),pruning_method=prune.L1Unstructured,amount=amount)
        #Rewind Weights
        for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
            with torch.no_grad():
                module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                module.weight_orig.copy_(module_rewind.weight)
        #Train Weights
        train(model, train_loader,num_epochs = num_epochs)
        
        #Log Results
        zero, total = sparsity_print(model)
        zeros.append(zero)
        totals.append(total)
        acc.append(test(model, test_loader))
        if save_model:
            torch.save(model.state_dict(), path + name + '_iter' + str(i+1))
            
        if reinit:
            #Rewind Weights
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_reinit = get_parameters_to_prune(reinit_model)[idx][0]
                    module.weight_orig.copy_(module_reinit.weight)
                    
            train(model, train_loader,num_epochs = num_epochs)
            reinit_acc.append(test(model, test_loader))
            
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                    module.weight_orig.copy_(module_rewind.weight)
        else:
            reinit_acc.append(0)
            
        np.save(path + name + '_log', np.array([acc,zeros,totals,reinit_acc]))
    
    pass
  
#Generate full sparsity curve with retraining for random pruning, this is not a method to compute a single, high-sparisty model with random pruning. This generates and trains models at all sparsities for comparison
def RandomPruning(model, name, path, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs = 60, amount = .2, save_model = True, seed = 0):
    torch.manual_seed(seed)
    zeros = []
    totals = []
    acc = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    
    model_rewind = copy.deepcopy(model).to(device)
    
    for i in range(start_iter,end_iter):
        print('Random Pruning Iteration:', i+1)
        #Prune
        prune.global_unstructured(get_parameters_to_prune(model),pruning_method=prune.RandomUnstructured,amount=amount)
        #Train Weights
        train(model, train_loader,num_epochs = num_epochs)
        
        #Log Results
        zero, total = sparsity_print(model)
        zeros.append(zero)
        totals.append(total)
        acc.append(test(model, test_loader))

            
        #Rewind Weights to save them
        for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
            with torch.no_grad():
                module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                module.weight_orig.copy_(module_rewind.weight)
                
        if save_model:
            torch.save(model.state_dict(), path + name + '_iter' + str(i+1))
            
        np.save(path + name + '_log', np.array([acc,zeros,totals]))
    
#Generate full sparsity curve to compare with LTR
def DistilledPruning(model, name, path, images_train, labels_train, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs_distilled = 1000, num_epochs_real = 60, k = 0, amount = .2, save_model = True, validate = False, seed = 0, reinit = False, reinit_model = None, distilled_lr = .01):
    torch.manual_seed(seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    accs = []
    zeros = []
    totals = []
    reinit_acc = []
    
    model_rewind = copy.deepcopy(model).to(device)
    torch.save(model.state_dict(), path + name + '_RewindWeights' + '_' + str(k))
    
    if k != 0:
        args = argparse.Namespace(lr_net=str(distilled_lr), device='cuda', epoch_eval_train=str(k),batch_train=512,dataset='cifar10',dsa=True,dsa_strategy='color_crop_cutout_flip_scale_rotate',dsa_param = ParamDiffAug(), dc_aug_param=None, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)) #, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)
        model_rewind, acc_train_list, acc_test = evaluate_synset(0, model_rewind,images_train,labels_train,test_loader,args)
        
    
    for i in range(start_iter,end_iter):
        print('Distilled Pruning Iteration ', i)
        args = argparse.Namespace(lr_net='.01', device='cuda', epoch_eval_train=str(num_epochs_distilled),batch_train=512,dataset='cifar10',dsa=True,dsa_strategy='color_crop_cutout_flip_scale_rotate',dsa_param = ParamDiffAug(), dc_aug_param=None, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)) #, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)
        #MTT Training on Distilled Data
        model, acc_train_list, acc_test = evaluate_synset(i+1, model,images_train,labels_train,test_loader,args)
        prune.global_unstructured(get_parameters_to_prune(model),pruning_method=prune.L1Unstructured,amount=amount)
        #Rewind Weights
        for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
            with torch.no_grad():
                module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                module.weight_orig.copy_(module_rewind.weight)
    
        if save_model:
            torch.save(model.state_dict(), path + name + '_iter' + str(i+1))
            
        if validate:
            train(model, train_loader,num_epochs = num_epochs_real)
            accs.append(test(model, test_loader))
            zero, total = sparsity_print(model)
            zeros.append(zero)
            totals.append(total)
            #Rewind Weights
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                    module.weight_orig.copy_(module_rewind.weight)
                    
            np.save(path + name + '_log', np.array([accs, zeros, totals, reinit_acc]))
        
        if reinit:
            #Rewind Weights to Reinit Model
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_reinit = get_parameters_to_prune(reinit_model)[idx][0]
                    module.weight_orig.copy_(module_reinit.weight)
                    
            train(model, train_loader,num_epochs = num_epochs_real)
            reinit_acc.append(test(model, test_loader))
            
            for idx, (module, _) in enumerate(get_parameters_to_prune(model)):
                with torch.no_grad():
                    module_rewind = get_parameters_to_prune(model_rewind)[idx][0]
                    module.weight_orig.copy_(module_rewind.weight)
            np.save(path + name + '_log', np.array([accs, zeros, totals, reinit_acc]))
        else:
            reinit_acc.append(0)
            
    if not validate:
        train(model, train_loader,num_epochs = num_epochs_real)
        acc = (test(model, test_loader))
        zero, total = sparsity_print(model)
        np.save(path + name + '_log', np.array([acc, zero, total, reinit]))
    
path = './model_results_cifar100/'

Distilled Training Params: .085 lr and 1300 epochs

In [10]:
model = get_network('ConvNetW128', 3, 100)
args = argparse.Namespace(lr_net=str(.09), device='cuda', epoch_eval_train=str(500),batch_train=100,dataset='cifar100',dsa=True,dsa_strategy='color_crop_cutout_flip_scale_rotate',dsa_param = ParamDiffAug(), dc_aug_param=None, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True))                        
for i in range(20):
    model, acc_train_list, acc_test = evaluate_synset(1, model,images_train,labels_train,test_loader,args)

100%|██████████| 501/501 [00:55<00:00,  9.03it/s]


[2023-05-30 16:23:45] Evaluate_01: epoch = 0500 train time = 55 s train loss = 0.007421 train acc = 1.0000, test acc = 0.3796


100%|██████████| 501/501 [00:53<00:00,  9.31it/s]


[2023-05-30 16:24:39] Evaluate_01: epoch = 0500 train time = 53 s train loss = 0.003408 train acc = 1.0000, test acc = 0.3957


100%|██████████| 501/501 [00:52<00:00,  9.49it/s]


[2023-05-30 16:25:32] Evaluate_01: epoch = 0500 train time = 52 s train loss = 0.004517 train acc = 1.0000, test acc = 0.3931


100%|██████████| 501/501 [00:54<00:00,  9.18it/s]


[2023-05-30 16:26:27] Evaluate_01: epoch = 0500 train time = 54 s train loss = 0.002383 train acc = 1.0000, test acc = 0.3780


 43%|████▎     | 217/501 [00:22<00:29,  9.60it/s]


KeyboardInterrupt: 

In [8]:
def objective(trial):
    params = {
              'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1),
              }

    torch.manual_seed(0)
    model = get_network('ConvNetW128', 3, 100)
    args = argparse.Namespace(lr_net=str(params['learning_rate']), device='cuda', epoch_eval_train=str(1000),batch_train=512,dataset='cifar100',dsa=True,dsa_strategy='color_crop_cutout_flip_scale_rotate',dsa_param = ParamDiffAug(), dc_aug_param=None, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True))                        
    model, acc_train_list, acc_test = evaluate_synset(1, model,images_train,labels_train,test_loader,args)

    return acc_test

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=20)

[I 2023-05-30 16:16:20,740] A new study created in memory with name: no-name-c41bfd89-1a61-49e3-b654-addf8e24482e
100%|██████████| 1001/1001 [01:12<00:00, 13.87it/s]
[I 2023-05-30 16:17:32,905] Trial 0 finished with value: 0.3678 and parameters: {'learning_rate': 0.026211558904925995}. Best is trial 0 with value: 0.3678.


[2023-05-30 16:17:32] Evaluate_01: epoch = 1000 train time = 72 s train loss = 0.012755 train acc = 1.0000, test acc = 0.3678


100%|██████████| 1001/1001 [01:12<00:00, 13.76it/s]
[I 2023-05-30 16:18:45,667] Trial 1 finished with value: 0.3886 and parameters: {'learning_rate': 0.08994542835725695}. Best is trial 1 with value: 0.3886.


[2023-05-30 16:18:45] Evaluate_01: epoch = 1000 train time = 72 s train loss = 0.001732 train acc = 1.0000, test acc = 0.3886


100%|██████████| 1001/1001 [01:12<00:00, 13.87it/s]
[I 2023-05-30 16:19:57,853] Trial 2 finished with value: 0.3878 and parameters: {'learning_rate': 0.06305983425758348}. Best is trial 1 with value: 0.3886.


...

[2023-05-30 16:21:09] Evaluate_01: epoch = 1000 train time = 72 s train loss = 0.002632 train acc = 1.0000, test acc = 0.3763


100%|██████████| 1001/1001 [01:12<00:00, 13.82it/s]
[I 2023-05-30 16:22:22,448] Trial 4 finished with value: 0.3874 and parameters: {'learning_rate': 0.09087063014134104}. Best is trial 1 with value: 0.3886.


[2023-05-30 16:22:22] Evaluate_01: epoch = 1000 train time = 72 s train loss = 0.003265 train acc = 1.0000, test acc = 0.3874


  8%|▊         | 82/1001 [00:05<01:06, 13.73it/s]
[W 2023-05-30 16:22:28,431] Trial 5 failed with parameters: {'learning_rate': 0.016780346092693223} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_32/3820575577.py", line 9, in objective
    model, acc_train_list, acc_test = evaluate_synset(1, model,images_train,labels_train,test_loader,args)
  File "/notebooks/mtt_distillation/utils.py", line 371, in evaluate_synset
    loss_train, acc_train = epoch('train', trainloader, net, optimizer, criterion, args, aug=True, texture=texture)
  File "/notebooks/mtt_distillation/utils.py", line 322, in epoch
    img = DiffAugment(img, args.dsa_strategy, param=args.dsa_param)
  File "/notebooks/mtt_distillation/utils.py", line 542, in DiffAugment
    x = f(x, param)
  File "/notebooks/mtt_distillation/utils

KeyboardInterrupt: 

In [14]:
model = get_network('ConvNetW128', 3, 100)
args = argparse.Namespace(lr_net='.09', device='cuda', epoch_eval_train=str(1300),batch_train=512,dataset='cifar100',dsa=True,dsa_strategy='color_crop_cutout_flip_scale_rotate',dsa_param = ParamDiffAug(), dc_aug_param=None, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)) #, zca_trans=kornia.enhance.ZCAWhitening(eps=0.1, compute_inv=True)
model, acc_train_list, acc_test = evaluate_synset(1, model,images_train,labels_train,test_loader,args)

100%|██████████| 1301/1301 [01:33<00:00, 13.92it/s]

[2023-05-30 16:31:14] Evaluate_01: epoch = 1300 train time = 93 s train loss = 0.005592 train acc = 1.0000, test acc = 0.3993


In [ ]:
for i in range(1,5):
    torch.manual_seed(i)
    model = get_network('ConvNetW128', 3, 100)
    DistilledPruning(model, 'dist_c100_ipc10_seed' + str(i), path, images_train, labels_train, train_loader, test_loader, start_iter = 0, end_iter = 20, num_epochs_distilled = 1250, num_epochs_real = 120, k = 0, amount = .2, save_model = False, validate = True, seed = 0, reinit = False, reinit_model = None, distilled_lr = .09)

Distilled Pruning Iteration  0


  0%|          | 0/1251 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 1251/1251 [00:55<00:00, 22.35it/s]


[2023-05-31 18:21:30] Evaluate_01: epoch = 1250 train time = 55 s train loss = 0.029523 train acc = 1.0000, test acc = 0.3745


/tmp/ipykernel_98/2071708352.py:197: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save(path + name + '_log', np.array([accs, zeros, totals, reinit_acc]))


Test Accuracy: 0.5176
Number of Zero Weights: 100634.0
Total Number of Weights: 503168.0
Sparsity 0.2000007949631137
Distilled Pruning Iteration  1


...

[2023-05-31 19:06:01] Evaluate_04: epoch = 1250 train time = 53 s train loss = 0.003360 train acc = 1.0000, test acc = 0.3712
Test Accuracy: 0.4791
Number of Zero Weights: 297070.0
Total Number of Weights: 503168.0
Sparsity 0.590399230475706
Distilled Pruning Iteration  4


100%|██████████| 1251/1251 [00:54<00:00, 22.94it/s]


[2023-05-31 19:20:54] Evaluate_05: epoch = 1250 train time = 54 s train loss = 0.003594 train acc = 1.0000, test acc = 0.3687
Test Accuracy: 0.4738
Number of Zero Weights: 338290.0
Total Number of Weights: 503168.0
Sparsity 0.6723201793436785
Distilled Pruning Iteration  5


100%|██████████| 1251/1251 [00:54<00:00, 22.92it/s]


[2023-05-31 19:35:49] Evaluate_06: epoch = 1250 train time = 54 s train loss = 0.014486 train acc = 1.0000, test acc = 0.3672


In [9]:
for i in range(1,2):
    torch.manual_seed(i)
    model = get_network('ConvNetW128', 3,10)
    LotteryTicketRewinding(model, 'ltr_c10_seed' + str(i), path, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs = 60, k = 1, amount = .2, save_model = False, seed = i, reinit = False)

LTR Iteration: 1
Number of Zero Weights: 63770.0
Total Number of Weights: 318848.0
Sparsity 0.20000125451625853
Test Accuracy: 0.8143
LTR Iteration: 2
Number of Zero Weights: 114786.0
Total Number of Weights: 318848.0
Sparsity 0.36000225812926534
Test Accuracy: 0.8132
LTR Iteration: 3
Number of Zero Weights: 155598.0
Total Number of Weights: 318848.0
Sparsity 0.48800055198715375
Test Accuracy: 0.8118
LTR Iteration: 4
Number of Zero Weights: 188248.0
Total Number of Weights: 318848.0
Sparsity 0.590400441589723
Test Accuracy: 0.8094
LTR Iteration: 5
Number of Zero Weights: 214368.0
Total Number of Weights: 318848.0
Sparsity 0.6723203532717784
Test Accuracy: 0.8101
LTR Iteration: 6
Number of Zero Weights: 235264.0
Total Number of Weights: 318848.0
Sparsity 0.7378562826174228
Test Accuracy: 0.806
LTR Iteration: 7
Number of Zero Weights: 251981.0
Total Number of Weights: 318848.0
Sparsity 0.7902856533520675
Test Accuracy: 0.8042
LTR Iteration: 8
Number of Zero Weights: 265354.0
Total Number

In [6]:
for i in range(1):
    torch.manual_seed(i)
    model = get_network('ConvNetW128', 3,100)
    RandomPruning(model, 'random_c10_seed' + str(i), path, train_loader, test_loader, start_iter = 0, end_iter = 30, num_epochs = 60, amount = .2, save_model = False, seed = i)

Random Pruning Iteration: 1
Number of Zero Weights: 100634.0
Total Number of Weights: 503168.0
Sparsity 0.2000007949631137
Test Accuracy: 0.8126
Random Pruning Iteration: 2
Number of Zero Weights: 181141.0
Total Number of Weights: 503168.0
Sparsity 0.3600010334520478
Test Accuracy: 0.7985
Random Pruning Iteration: 3
Number of Zero Weights: 245547.0
Total Number of Weights: 503168.0
Sparsity 0.4880020192063088
Test Accuracy: 0.7913
Random Pruning Iteration: 4
Number of Zero Weights: 297070.0
Total Number of Weights: 503168.0
Sparsity 0.590399230475706
Test Accuracy: 0.781
Random Pruning Iteration: 5
Number of Zero Weights: 338290.0
Total Number of Weights: 503168.0
Sparsity 0.6723201793436785
Test Accuracy: 0.7751
Random Pruning Iteration: 6
Number of Zero Weights: 371266.0
Total Number of Weights: 503168.0
Sparsity 0.7378569384380564
Test Accuracy: 0.7669
Random Pruning Iteration: 7
Number of Zero Weights: 397646.0
Total Number of Weights: 503168.0
Sparsity 0.7902847557873315
Test Accu